In [1]:
import sys
print(sys.executable)

/Users/billdeng/anaconda3/envs/de_novo/bin/python


In [2]:
from Bio import SeqIO

First, read and visualize input and output data.

In [3]:
# Path to FASTA file
fasta_file = 'ML/input/input.fa'

# Initialize an empty dictionary to store sequences
chromosomes = {}

# Using SeqIO.parse to read the FASTA file
for record in SeqIO.parse(fasta_file, "fasta"):
    chromosome_name = record.id
    sequence = str(record.seq)
    chromosomes[chromosome_name] = sequence

# Now, chromosomes dictionary contains all the sequences
# For example, to access the sequence for chromosome 1, you can use:
# chromosome_1_sequence = chromosomes['1']

# If you want to print out the names of all chromosomes read, you can do:
# print("Chromosomes read from the FASTA file:")
# for chromosome in chromosomes.keys():
#     print(chromosome)

FileNotFoundError: [Errno 2] No such file or directory: 'ML/input/input.fa'

In [4]:
for chromosome in chromosomes.keys():
    print(chromosome)
    # print(type(chromosome))  # This will print the type of the chromosome variable

In [5]:
print(type(chromosomes[chromosome_name]))

NameError: name 'chromosome_name' is not defined

In [6]:
chromosome_name = "chr22"  # Just an example, replace it with your actual key
# Print the type of the object and a part of the object (if it's a string, for example)
print(f"Type: {type(chromosomes[chromosome_name])}, Beginning of Content: '{chromosomes[chromosome_name][:10]}'")
print(f"Type: {len(chromosomes[chromosome_name])}, Beginning of Content: '{chromosomes[chromosome_name][1211231:1211252]}'")

KeyError: 'chr22'

Use the de novo environment.

In [7]:
import os
import numpy as np
import tensorflow

2024-04-07 16:24:33.009699: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
from keras import Model
from keras import backend as K
from keras.layers import Conv2D

from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input

In [9]:
from janggu import *

ModuleNotFoundError: No module named 'keras.layers.wrappers'

In [ ]:
from janggu.data import Bioseq
from janggu.data import Cover
from janggu.data import ReduceDim
from janggu.layers import DnaConv2D

from sklearn.metrics import roc_aucgit_score

In [ ]:
from pkg_resources import resource_filename